In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# patching
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from utils.utils import find_nearest

In [2]:
path = Path(".").resolve().parent.parent / "data" / "sensors_characteristics"

In [3]:
path

WindowsPath('C:/Users/fmark/PycharmProjects/dissertation/data/sensors_characteristics')

In [4]:
reference_voltage = 3.3
steps = 2**12

In [5]:
steps

4096

In [6]:
step = reference_voltage / steps

In [7]:
step

0.0008056640625

In [8]:
codes = [i for i in range(steps)]
voltages = [(code / steps) * reference_voltage + step for code in codes]
df_mcp_4725 = pd.DataFrame({"code": codes, "voltage": voltages})

In [9]:
df_mcp_4725.head()

,code,voltage
0,0,0.000806
1,1,0.001611
2,2,0.002417
3,3,0.003223
4,4,0.004028


In [10]:
df_mcp_4725["divided_voltage"] = df_mcp_4725["voltage"] / 100

In [11]:
df_mcp_4725["divided_voltage (mV)"] = df_mcp_4725["divided_voltage"] * 1000

In [12]:
df_mcp_4725

,code,voltage,divided_voltage,divided_voltage (mV)
0,0,0.000806,0.000008,0.008057
1,1,0.001611,0.000016,0.016113
2,2,0.002417,0.000024,0.024170
3,3,0.003223,0.000032,0.032227
4,4,0.004028,0.000040,0.040283
...,...,...,...,...
4091,4091,3.296777,0.032968,32.967773
4092,4092,3.297583,0.032976,32.975830
4093,4093,3.298389,0.032984,32.983887
4094,4094,3.299194,0.032992,32.991943


In [13]:
sensor_characteristics = pd.read_csv(path / "ТЖК (J).csv",index_col="temp")

In [14]:
sensor_characteristics.loc[0:10]

,value
temp,
0,0.000
1,0.050
2,0.101
3,0.151
4,0.202
5,0.253
6,0.303
7,0.354
8,0.405


In [15]:
df = pd.DataFrame()

In [16]:
df

""


In [17]:
min_temp, max_temp = 0, 100

In [18]:
min_value = sensor_characteristics.loc[min_temp]["value"]
max_value = sensor_characteristics.loc[max_temp]["value"]

In [19]:
min_value

0.0

In [20]:
max_value

5.269

In [21]:
df["voltage"] = sensor_characteristics[sensor_characteristics.index.isin(range(min_temp, max_temp + 1))]["value"]

In [22]:
df.head(10)

,voltage
temp,
0,0.000
1,0.050
2,0.101
3,0.151
4,0.202
5,0.253
6,0.303
7,0.354
8,0.405


In [23]:
_simulation_range = [value for value in df_mcp_4725["divided_voltage (mV)"] if min_value < value < max_value]

In [24]:
_simulation_range[-10:]

[5.188476562500001,
 5.196533203125,
 5.20458984375,
 5.212646484375,
 5.220703125,
 5.228759765625,
 5.23681640625,
 5.244873046875,
 5.2529296875,
 5.260986328125]

In [25]:
df["calc"] = df["voltage"].apply(lambda x: find_nearest(_simulation_range, x))

In [26]:
df

,voltage,calc
temp,,
0,0.000,0.008057
1,0.050,0.048340
2,0.101,0.104736
3,0.151,0.153076
4,0.202,0.201416
...,...,...
96,5.052,5.051514
97,5.106,5.107910
98,5.160,5.156250


In [39]:
df["code"] = df_mcp_4725["te"].apply(lambda x: df_mcp_4725[df_mcp_4725["divided_voltage (mV)"] == df.iloc[x]["calc"]]["code"], axis=0)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
             91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
           dtype='int64', name='temp', length=101)

In [55]:
df["calc"].apply(lambda x: int(df_mcp_4725[df_mcp_4725["divided_voltage (mV)"] == x].iloc[0]["code"]))

temp
0        0
1        5
2       12
3       18
4       24
      ... 
96     626
97     633
98     639
99     646
100    652
Name: calc, Length: 101, dtype: int64

In [52]:
df_mcp_4725[df_mcp_4725["divided_voltage (mV)"] == df["calc"][100]]["code"]

652    652
Name: code, dtype: int64

In [51]:
df["calc"].apply(lambda x: x)

temp
0      0.008057
1      0.048340
2      0.104736
3      0.153076
4      0.201416
         ...   
96     5.051514
97     5.107910
98     5.156250
99     5.212646
100    5.260986
Name: calc, Length: 101, dtype: float64